## STEPS TO BE FOLLOWED 

- Notebook to Create Connection Between Databricks to MYSQL and BigQuery
- Reads the conig_table rows for **active_flag=1 AND load_flag=1**
- Export the Each **BigQuery Table to --> GCS Stagging Layer**
- Update and Log the status of the Load in config_table

In [0]:
dbutils.widgets.text("mysql_host","136.113.73.13")
mysql_host=dbutils.widgets.get("mysql_host")

dbutils.widgets.text("mysql_port","3306")
mysql_port=dbutils.widgets.get("mysql_port")

dbutils.widgets.text("mysql_user","root")
mysql_user=dbutils.widgets.get("mysql_user")

dbutils.widgets.text("mysql_password","Admin@1234")
mysql_password=dbutils.widgets.get("mysql_password")

dbutils.widgets.text("bq_sa_key","/Workspace/Users/anurag.srivastava@koantekorg.onmicrosoft.com/Migrationproject/datamigrationproject-475415-ffc1c7e9a773.json")
bq_sa_key=dbutils.widgets.get("bq_sa_key")

In [0]:
%pip install mysql-connector-python google-cloud-bigquery


In [0]:
import json,os,datetime as dt
import mysql.connector as mc
from google.cloud import bigquery
from contextlib import contextmanager

#Enclosing it in Dictiornary
MYSQL={
    'host':mysql_host.strip(),
    'port':int(mysql_port),
    'user':mysql_user.strip(),
    'pwd':mysql_password
}

#Repalcing the dbfs:/ with /dbfs/`
BQ_KEY=bq_sa_key.replace('dbfs:/','/dbfs/')

In [0]:
@contextmanager
def mysql_conn():
    conn=mc.connect(
            host=MYSQL['host'],port=MYSQL['port'],
            user=MYSQL['user'],password=MYSQL['pwd']
    )
    try:
        yield conn 
    except Exception as e:
        print(f"Failed to Connect to MYSQL \nHere is the Error -: \n{e}")
    finally:
        conn.close()



In [0]:
def fetch_eligible_tables():
    with mysql_conn() as conn:
        cur=conn.cursor()
        cur.execute("""SELECT * FROM 
                    GCPMigrationMeta.config_table 
                    where active_flag=1 and load_flag=1""")
        rows= cur.fetchall()
        cur.close()
    return rows


In [0]:
def set_bq_to_gcs_status(table_name,status,error=None):
    with mysql_conn() as conn:
        cur=conn.cursor()

        if status=='IN_PROGRESS':
            cur.execute("""
                        UPDATE GCPMigrationMeta.config_table 
                        SET bq_to_gcs_status='IN_PROGRESS',
                        last_run_ts=NOW(),
                        error_message=NULL
                        WHERE table_name=%s
                        """,(table_name))

        elif status=='COMPLETED':
            cur.execute("""
                        UPDATE GCPMigrationMeta.config_table 
                        SET bq_to_gcs_status='COMPLETED',
                        last_run_ts=NOW(),
                        error_message=NULL
                        WHERE table_name=%s
                        """,(table_name))


        else:    
            cur.execute("""
                        UPDATE GCPMigrationMeta.config_table 
                        SET bq_to_gcs_status='FAILED',
                        error_message=%s
                        WHERE table_name=%s
                        """,(str(err)[:2000]if err else "FAILED" ,table_name))
        conn.commit()
        cur.close()

In [0]:
def bq_client():
    os.environ['GOOLGE_APPLICATION_CREDENTIALS']=BQ_KEY
    with open(BQ_KEY) as f: 
        proj=json.load(f).get("project_id")
    return bigquery.Client(project=proj)    

In [0]:
def export_table_to_gcs(source_project,source_dataset,table_name,gcs_base):
    """
    Exporting the BigQuery table GCS Bucket 
    Aurthor : Anurag Srivastava
    """